In [1]:
import os
from bs4 import BeautifulSoup
import concurrent.futures
import time
f_list = os.listdir("wiki")
f_list

['Furubira_District,_Hokkaido.html',
 'Valentin_Yanin.html',
 'Kings_XI_Punjab_in_2014.html',
 'William_Harvey_Lillard.html',
 'Radial_Road_3.html',
 'George_Weldrick.html',
 'Zgornji_Otok.html',
 'Blue_Heelers_(season_8).html',
 'Taggen_Nunatak.html',
 '1951_National_League_tie-breaker_series.html',
 'List_of_number-one_singles_of_1993_(Finland).html',
 'Vrila.html',
 'William_Henry_Porter.html',
 'Clive_Brown_(footballer).html',
 '2010_Karshi_Challenger_%E2%80%93_Singles.html',
 'Blick_nach_Rechts.html',
 'Central_District_(Rezvanshahr_County).html',
 'Gal%C3%A1pagos,_Guadalajara.html',
 'Campus_of_Texas_A%26M_University.html',
 'Alexios_Aspietes.html',
 'Mei_Lanfang.html',
 'Thalkirchen-Obersendling-Forstenried-F%C3%BCrstenried-Solln.html',
 'Coalville_Town_railway_station.html',
 'Gennady_Lesun.html',
 'Bartrum_Glacier.html',
 'Victor_S._Mamatey.html',
 'Gottfried_Keller.html',
 'Table_Point_Formation.html',
 'Nobuhiko_Ushiba.html',
 'Master_of_Space_and_Time.html',
 'Early_medieva

In [2]:
len(os.listdir("wiki"))

999

In [3]:
with open("wiki/Supermoon.html") as f:
    print(f.read())

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Supermoon - Wikipedia</title>
<script>document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );</script>
<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Supermoon","wgTitle":"Supermoon","wgCurRevisionId":768351483,"wgRevisionId":768351483,"wgArticleId":31156672,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia articles scheduled for update tagging","All articles with unsourced statements","Articles with unsourced statements from September 2015","Commons category with page title same as on Wikidata","Articles containing video clips","Astrology","Astronomical events","Lunar science","Moon"],"wgBreakFrames":false,"wgPageContentLanguage

In [4]:
pool = concurrent.futures.ProcessPoolExecutor(max_workers=5)

def read_data(filename):
    with open(filename) as f:
        data = f.read()
    return data

start = time.time()
filenames = ["wiki/{}".format(f) for f in os.listdir("wiki")]
content = pool.map(read_data, filenames)
content = list(content)

end = time.time()
print(end - start)
articles = [f.replace(".html", "").replace("wiki/", "") for f in filenames]

0.9686994552612305


In [8]:
def parse_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return str(soup.find_all("div", id="content")[0])

start = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=2)
parsed = pool.map(parse_html, content)
parsed = list(parsed)
end = time.time()
print(end - start)

52.52501606941223


In [14]:
def count_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    tags = {}
    for tag in soup.find_all():
        if tag.name not in tags:
            tags[tag.name] = 0
        tags[tag.name] += 1
    return tags

start = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
tags = pool.map(count_tags, parsed)
tags = list(tags)

tag_counts = {}
for tag in tags:
    for k,v in tag.items():
        if k not in tag_counts:
            tag_counts[k] = 0
        tag_counts[k] += v
end = time.time()

print(end - start)
tag_counts

27.626102447509766


{'a': 161065,
 'abbr': 3665,
 'annotation': 2,
 'area': 39,
 'audio': 2,
 'b': 14455,
 'bdi': 4,
 'big': 75,
 'blockquote': 58,
 'br': 4986,
 'caption': 200,
 'center': 64,
 'cite': 3563,
 'code': 108,
 'dd': 1376,
 'del': 2,
 'div': 28581,
 'dl': 457,
 'dt': 334,
 'font': 40,
 'h1': 999,
 'h2': 4045,
 'h3': 777,
 'h4': 117,
 'h5': 4,
 'h6': 1,
 'hr': 51,
 'i': 18246,
 'img': 6701,
 'li': 85779,
 'map': 2,
 'math': 2,
 'mo': 2,
 'mrow': 2,
 'mstyle': 2,
 'noscript': 999,
 'ol': 858,
 'p': 7998,
 'pre': 1,
 'q': 76,
 'rb': 16,
 'rp': 32,
 'rt': 16,
 'ruby': 16,
 's': 10,
 'samp': 2,
 'semantics': 2,
 'small': 3272,
 'source': 2,
 'span': 67350,
 'strong': 599,
 'sub': 151,
 'sup': 11157,
 'table': 4010,
 'td': 57673,
 'th': 14472,
 'tr': 27300,
 'u': 51,
 'ul': 10972,
 'wbr': 85}

In [15]:
from collections import Counter
import re

def count_words(html):
    soup = BeautifulSoup(html, 'html.parser')
    words = {}
    text = soup.get_text()
    text = re.sub("\W+", " ", text.lower())
    words = text.split(" ")
    words = [w for w in words if len(w) >= 5]
    return Counter(words).most_common(10)

start = time.time()
pool = concurrent.futures.ProcessPoolExecutor(max_workers=3)
words = pool.map(count_words, parsed)
words = list(words)

word_counts = {}
for wc in words:
    for word, count in wc:
        if word not in word_counts:
            word_counts[word] = 0
        word_counts[word] += 1
end = time.time()

print(end - start)
word_counts

26.8257737159729


{'gibson': 1,
 'terms': 1,
 'assist': 1,
 'charlton': 1,
 'fellow': 1,
 'nurpur': 1,
 'touyour': 1,
 'medicago': 1,
 'jensen': 1,
 'thessalonica': 1,
 'playing': 3,
 'industry': 1,
 'niigata': 2,
 'renault': 2,
 'villeneuve': 1,
 'mafia': 2,
 'manager': 1,
 'monks': 1,
 'clube': 1,
 'strategic': 1,
 'please': 5,
 'landscape': 1,
 '91306': 1,
 'jackson': 1,
 'idaho': 1,
 'blake': 1,
 'nepal': 2,
 'lanfang': 1,
 'patch': 1,
 'tadashi': 1,
 'franja': 1,
 'acrolophus': 1,
 'limited': 3,
 'female': 2,
 'cattle': 1,
 'hunter': 2,
 'playback': 1,
 'lesbos': 1,
 'soccer': 1,
 'staff': 1,
 'motion': 1,
 'epropetes': 1,
 'frederick': 1,
 'tramery': 1,
 'insulin': 1,
 'kabud': 1,
 'martinus': 1,
 'protocols': 1,
 'gijón': 1,
 'newton': 2,
 '0000may': 1,
 'otology': 1,
 'sheehan': 1,
 'banovinas': 1,
 'chips': 1,
 'mykines': 1,
 'riding': 1,
 'phipps': 1,
 'recreation': 1,
 'pupils': 1,
 'peoplebodoland': 1,
 'division': 12,
 'pempheridae': 1,
 'warrington': 1,
 'serena': 1,
 'tulem': 1,
 'encrypt